![display relevant image here](path/url/to/image)
- Banner/header image

# Title
- Relevant to Data and Business Context

## Overview
- BLUF (Bottom Line Up Front)
- One paragraph summary of findings and analysis
- Frame your 'story'

## Business Understanding
- Set the stage for analysis
- Why are these findings relevant/important?
- Introduce stakeholders
- Postulate about questions you want to ask/answer

## Data Understanding
- Present the source of data
- Describe the data available
- What is relevant to keep what is not
- Present any data cleaning that needs to happen
- Null values? Type mismatches? Duplicates?

In [10]:
# Imports
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Google Play Dataset") \
    .getOrCreate()

df = spark.read.option("header", "true").option("inferSchema", "true").csv("google_play_store_dataset.csv")


# verify Spark session

if (spark.getActiveSession()):
    print('yes')
else:
    print('no')

yes


Prompts:
Im working for a company that wants to know some general trends in the android/google store market. I need help on analyzing and displaying (dashboards) general trends in the app success across the google play store.
I can only use Pyspark for now. Do not update my notebook. Just show me the code so I can inspect it.

2nd prompt:I want this code take out the null values and remove any duplicates. Also, What’s the average rating and install count across different content ratings (Everyone, Teen, Mature 17+, etc.)? using only pyspark

In [11]:
# EDA Code Here - Create New Cells As Needed
# Basic data exploration
df.printSchema()
df.show(5)
print(f"Total records: {df.count()}")
print(f"Total columns: {len(df.columns)}")

# Check for missing values
from pyspark.sql.functions import col, isnan, when, count
df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns]).show()




root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|    Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+----------------+--------------

In [12]:

# Clean and prepare data


# Remove nulls and duplicates
df_clean = df.dropna() \
    .dropDuplicates() \
    .filter(col("Rating").isNotNull()) \
    .filter(col("Installs") != "Free") \
    .withColumn("Installs_Clean", regexp_replace(col("Installs"), "[+,]", "")) \
    .withColumn("Installs_Numeric", col("Installs_Clean").cast("long")) \
    .withColumn("Price_Numeric", regexp_replace(col("Price"), "[$]", "").cast("double")) \
    .withColumn("Reviews_Numeric", col("Reviews").cast("long"))

print(f"Original records: {df.count()}")
print(f"After cleaning: {df_clean.count()}")


Original records: 10841
After cleaning: 10356


In [13]:
# 1. Top Categories by Success
category_success = df_clean.groupBy("Category") \
    .agg(
        avg("Rating").alias("Avg_Rating"),
        sum("Installs_Numeric").alias("Total_Installs"),
        count("*").alias("App_Count")
    ) \
    .orderBy(desc("Total_Installs"))

category_success.show(20)

# 2. Rating Distribution
rating_dist = df_clean.groupBy("Rating").count().orderBy("Rating")
rating_dist.show()

# 3. Free vs Paid Performance
pricing_analysis = df_clean.withColumn("App_Type", 
    when(col("Price_Numeric") == 0, "Free").otherwise("Paid")) \
    .groupBy("App_Type") \
    .agg(
        count("*").alias("Count"),
        avg("Rating").alias("Avg_Rating"),
        avg("Installs_Numeric").alias("Avg_Installs")
    )

pricing_analysis.show()

# 4. Top Apps by Install Count
top_apps = df_clean.select("App", "Category", "Rating", "Installs_Numeric") \
    .orderBy(desc("Installs_Numeric")) \
    .limit(20)

top_apps.show()



+-------------------+-----------------+--------------+---------+
|           Category|       Avg_Rating|Total_Installs|App_Count|
+-------------------+-----------------+--------------+---------+
|               GAME|              NaN|   31544024415|     1121|
|      COMMUNICATION|              NaN|   24152276251|      366|
|             SOCIAL|              NaN|   12513867902|      280|
|       PRODUCTIVITY|              NaN|   12463091369|      407|
|              TOOLS|              NaN|   11452771915|      843|
|             FAMILY|              NaN|   10041692505|     1943|
|        PHOTOGRAPHY|              NaN|    9721247655|      322|
|   TRAVEL_AND_LOCAL|              NaN|    6361887146|      237|
|      VIDEO_PLAYERS|              NaN|    6222002720|      175|
| NEWS_AND_MAGAZINES|              NaN|    5393217760|      264|
|           SHOPPING|              NaN|    2573348785|      224|
|      ENTERTAINMENT|4.136036036036035|    2455660000|      111|
|    PERSONALIZATION|    

In [14]:
# Average rating and install count by content rating
content_rating_analysis = df_clean.groupBy("Content Rating") \
    .agg(
        avg("Rating").alias("Avg_Rating"),
        avg("Installs_Numeric").alias("Avg_Installs"),
        count("*").alias("App_Count"),
        sum("Installs_Numeric").alias("Total_Installs")
    ) \
    .orderBy(desc("Avg_Rating"))

content_rating_analysis.show()

# Safe printing with null checks
print("=== CONTENT RATING PERFORMANCE ===")
content_data = content_rating_analysis.collect()
for row in content_data:
    content_rating = row['Content Rating'] if row['Content Rating'] else "Unknown"
    avg_rating = row['Avg_Rating'] if row['Avg_Rating'] else 0
    avg_installs = row['Avg_Installs'] if row['Avg_Installs'] else 0
    app_count = row['App_Count'] if row['App_Count'] else 0
    
    print(f"{content_rating}: {avg_rating:.2f} avg rating, {avg_installs:,.0f} avg installs, {app_count} apps")



+---------------+----------+--------------------+---------+--------------+
| Content Rating|Avg_Rating|        Avg_Installs|App_Count|Total_Installs|
+---------------+----------+--------------------+---------+--------------+
|        Unrated|       NaN|             25250.0|        2|         50500|
|           Teen|       NaN|2.6979302262652706E7|     1146|   30918280393|
|     Mature 17+|       NaN|   9618154.434977578|      446|    4289696878|
|   Everyone 10+|       NaN|2.9685071074270558E7|      377|   11191271795|
|       Everyone|       NaN|1.1959858587231504E7|     8380|  100223614961|
|Adults only 18+|       4.3|   666666.6666666666|        3|       2000000|
|     5,000,000+|      NULL|                 4.0|        1|             4|
|     1,000,000+|      NULL|                 4.0|        1|             4|
+---------------+----------+--------------------+---------+--------------+

=== CONTENT RATING PERFORMANCE ===
Mature 17+: nan avg rating, 9,618,154 avg installs, 446 apps
Eve

In [15]:
# Top categories by content rating
category_content_analysis = df_clean.groupBy("Category", "Content Rating") \
    .agg(
        avg("Rating").alias("Avg_Rating"),
        count("*").alias("App_Count")
    ) \
    .orderBy("Category", desc("Avg_Rating"))

category_content_analysis.show(50)

# Content rating distribution
content_distribution = df_clean.groupBy("Content Rating") \
    .count() \
    .orderBy(desc("count"))

content_distribution.show()


+-------------------+---------------+------------------+---------+
|           Category| Content Rating|        Avg_Rating|App_Count|
+-------------------+---------------+------------------+---------+
|       traffic jams|     5,000,000+|              NULL|        1|
|     ART_AND_DESIGN|       Everyone|               NaN|       61|
|     ART_AND_DESIGN|   Everyone 10+|               4.7|        1|
|     ART_AND_DESIGN|           Teen| 4.466666666666666|        3|
|  AUTO_AND_VEHICLES|       Everyone|               NaN|       83|
|  AUTO_AND_VEHICLES|   Everyone 10+|               4.3|        1|
|  AUTO_AND_VEHICLES|           Teen|               4.2|        1|
|             BEAUTY|   Everyone 10+|               NaN|        2|
|             BEAUTY|       Everyone|               NaN|       45|
|             BEAUTY|           Teen|               NaN|        3|
|             BEAUTY|     Mature 17+|               NaN|        3|
|BOOKS_AND_REFERENCE|           Teen|               NaN|      

In [16]:
# Collect data for dashboard (small datasets only)
category_data = category_success.collect()
pricing_data = pricing_analysis.collect()
rating_data = rating_dist.collect()

# Display summary statistics
print("=== CATEGORY PERFORMANCE ===")
for row in category_data[:10]:
    print(f"{row['Category']}: {row['Total_Installs']:,} installs, {row['Avg_Rating']:.2f} rating")

print("\n=== PRICING ANALYSIS ===")
for row in pricing_data:
    print(f"{row['App_Type']}: {row['Count']} apps, {row['Avg_Rating']:.2f} avg rating")



=== CATEGORY PERFORMANCE ===
GAME: 31,544,024,415 installs, nan rating
COMMUNICATION: 24,152,276,251 installs, nan rating
SOCIAL: 12,513,867,902 installs, nan rating
PRODUCTIVITY: 12,463,091,369 installs, nan rating
TOOLS: 11,452,771,915 installs, nan rating
FAMILY: 10,041,692,505 installs, nan rating
PHOTOGRAPHY: 9,721,247,655 installs, nan rating
TRAVEL_AND_LOCAL: 6,361,887,146 installs, nan rating
VIDEO_PLAYERS: 6,222,002,720 installs, nan rating
NEWS_AND_MAGAZINES: 5,393,217,760 installs, nan rating

=== PRICING ANALYSIS ===
Free: 9589 apps, nan avg rating
Paid: 767 apps, nan avg rating


AI response:
This code provides:
Data cleaning for accurate analysis
Category performance metrics
Success scoring algorithm
Market trend identification
Dashboard-ready data exports
Correlation analysis between app characteristics and success
The cleaned data and summary tables can be directly imported into Tableau or other dashboard tools for visualization.

## Data Preparation



Promp:
Ok now we can integrate Pandas and Pyspark for the data preparation. I still want to analyzed well and display general trends of the google app.Write code to detect and remove duplicate apps based on the 'App' column.How can I validate that all values in the 'Reviews' column are numeric using PySpark?Create a new column in PySpark that indicates whether an app is free or paid as a binary 1/0 column.

In [17]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

# 1. Remove duplicates based on 'App' column
print(f"Before duplicate removal: {df.count()}")
df_no_duplicates = df.dropDuplicates(['App'])
print(f"After duplicate removal: {df_no_duplicates.count()}")

# 2. Validate Reviews column is numeric
reviews_validation = df_no_duplicates.select(
    count("*").alias("total_rows"),
    count(when(col("Reviews").cast("long").isNull(), 1)).alias("non_numeric_reviews")
)
reviews_validation.show()

# 3. Create binary free/paid column and clean data
df_clean = df_no_duplicates.withColumn("Price_Numeric", 
    regexp_replace(col("Price"), "[$]", "").cast("double")) \
    .withColumn("Is_Paid", when(col("Price_Numeric") > 0, 1).otherwise(0)) \
    .withColumn("Installs_Clean", regexp_replace(col("Installs"), "[+,]", "")) \
    .withColumn("Installs_Numeric", col("Installs_Clean").cast("long")) \
    .withColumn("Reviews_Numeric", col("Reviews").cast("long")) \
    .dropna()

# 4. Category Performance Analysis
category_trends = df_clean.groupBy("Category") \
    .agg(
        count("*").alias("App_Count"),
        avg("Rating").alias("Avg_Rating"),
        sum("Installs_Numeric").alias("Total_Installs"),
        avg("Is_Paid").alias("Paid_Ratio")
    ) \
    .orderBy(desc("Total_Installs"))

# 5. Free vs Paid Analysis
pricing_trends = df_clean.groupBy("Is_Paid") \
    .agg(
        count("*").alias("Count"),
        avg("Rating").alias("Avg_Rating"),
        avg("Installs_Numeric").alias("Avg_Installs")
    )

# 6. Content Rating Analysis
content_rating_trends = df_clean.groupBy("Content Rating") \
    .agg(
        count("*").alias("App_Count"),
        avg("Rating").alias("Avg_Rating"),
        avg("Installs_Numeric").alias("Avg_Installs")
    ) \
    .orderBy(desc("App_Count"))

# Show results
category_trends.show(10)
pricing_trends.show()
content_rating_trends.show()


Before duplicate removal: 10841
After duplicate removal: 9660
+----------+-------------------+
|total_rows|non_numeric_reviews|
+----------+-------------------+
|      9660|                  3|
+----------+-------------------+

+------------------+---------+----------+--------------+--------------------+
|          Category|App_Count|Avg_Rating|Total_Installs|          Paid_Ratio|
+------------------+---------+----------+--------------+--------------------+
|              GAME|      959|       NaN|   13878924415| 0.08550573514077164|
|     COMMUNICATION|      315|       NaN|   11038276251| 0.08571428571428572|
|             TOOLS|      827|       NaN|    8001771915|   0.094316807738815|
|      PRODUCTIVITY|      374|       NaN|    5793091369|  0.0748663101604278|
|            SOCIAL|      239|       NaN|    5487867902|0.012552301255230125|
|       PHOTOGRAPHY|      281|       NaN|    4649147655| 0.06761565836298933|
|            FAMILY|     1832|       NaN|    4427941505| 0.09989082969

In [18]:
# Convert to Pandas DataFrames
category_df = category_trends.toPandas()
pricing_df = pricing_trends.toPandas()
content_df = content_rating_trends.toPandas()

# Display insights
print("=== TOP CATEGORIES ===")
print(category_df.head(5)[['Category', 'Total_Installs', 'Avg_Rating']])

print("\n=== FREE VS PAID ===")
pricing_df['App_Type'] = pricing_df['Is_Paid'].map({0: 'Free', 1: 'Paid'})
print(pricing_df[['App_Type', 'Count', 'Avg_Rating']])

print("\n=== CONTENT RATINGS ===")
print(content_df.head()[['Content Rating', 'App_Count', 'Avg_Rating']])


=== TOP CATEGORIES ===
        Category  Total_Installs  Avg_Rating
0           GAME     13878924415         NaN
1  COMMUNICATION     11038276251         NaN
2          TOOLS      8001771915         NaN
3   PRODUCTIVITY      5793091369         NaN
4         SOCIAL      5487867902         NaN

=== FREE VS PAID ===
  App_Type  Count  Avg_Rating
0     Paid    756         NaN
1     Free   8900         NaN

=== CONTENT RATINGS ===
    Content Rating  App_Count  Avg_Rating
0         Everyone       7901         NaN
1             Teen       1036         NaN
2       Mature 17+        392         NaN
3     Everyone 10+        322         NaN
4  Adults only 18+          3         4.3


In [19]:
# Create success score combining multiple factors
df_success = df_clean.withColumn("Success_Score",
    (col("Rating") * 0.3) + 
    (log10(col("Installs_Numeric") + 1) * 0.4) + 
    (log10(col("Reviews_Numeric") + 1) * 0.3)
)

# Top performing apps
top_apps = df_success.select("App", "Category", "Rating", "Installs_Numeric", 
                            "Is_Paid", "Success_Score") \
    .orderBy(desc("Success_Score")) \
    .limit(20)

top_apps.show()

# Convert to Pandas for final dashboard export
dashboard_data = df_success.select(
    "App", "Category", "Rating", "Reviews_Numeric", "Installs_Numeric",
    "Price_Numeric", "Is_Paid", "Content Rating", "Success_Score"
).toPandas()

print(f"Dashboard dataset ready: {len(dashboard_data)} apps")
print(dashboard_data.head())


+--------------------+-------------------+------+----------------+-------+-------------+
|                 App|           Category|Rating|Installs_Numeric|Is_Paid|Success_Score|
+--------------------+-------------------+------+----------------+-------+-------------+
|          ABG Master|            MEDICAL|   NaN|              50|      1|          NaN|
|AI Image Recogniz...|        PHOTOGRAPHY|   NaN|             500|      0|          NaN|
|         ABTO CY-ET1|              TOOLS|   NaN|             100|      0|          NaN|
|     ABAI CE Scanner|             FAMILY|   NaN|            1000|      0|          NaN|
|  AC CE BJT Actorial|             FAMILY|   NaN|              50|      1|          NaN|
|"""i DT"" Fútbol....|             SPORTS|   NaN|             500|      0|          NaN|
|               AC-BL|      COMMUNICATION|   NaN|              50|      0|          NaN|
|"The FN ""Baby"" ...|BOOKS_AND_REFERENCE|   NaN|              10|      1|          NaN|
|   ACCEPT CE MARKING

In [22]:
# Save cleaned data file for Tableau usage
# Select only cleaned columns for Tableau
tableau_df = df_clean.select(
    "App",
    "Category", 
    "Rating",
    "Reviews_Numeric",
    "Installs_Numeric",
    "Size",
    "Price_Numeric",
    "Is_Paid",
    "Content Rating",
    "Genres",
    "Last Updated",
    "Current Ver",
    "Android Ver"
)

# Convert to Pandas and save as CSV
tableau_pandas = tableau_df.toPandas()
tableau_pandas.to_csv("google_play_store_cleaned.csv", index=False)

print(f"Saved {len(tableau_pandas)} cleaned records to google_play_store_cleaned.csv")
print("Columns saved:", tableau_pandas.columns.tolist())


Saved 9656 cleaned records to google_play_store_cleaned.csv
Columns saved: ['App', 'Category', 'Rating', 'Reviews_Numeric', 'Installs_Numeric', 'Size', 'Price_Numeric', 'Is_Paid', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [23]:
spark.stop()  # Stop the Spark session when done

## Data Analysis

In [21]:
# Analysis Code Here - if needed
None

### Link to Published Dashboard

## Conclusion

Markdown here